In [ ]:
import pandas as pd
import datetime as dt
import re as re
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ddid = pd.read_excel(r"C:\Users\emmaj\Documents\NSS\Python\Projects\ddid-rasberry-pi\data\TEIS Exit Data for NSS.xlsx", sheet_name = 3)
pd.set_option('display.max_colwidth', None)
ddid

In [ ]:
ddid = pd.DataFrame(ddid)
pd.set_option('display.max_colwidth', None)
ddid.head()

ddid_mvp = ddid[['Child ID', 'DOB', 'Initial IFSP Date', 'Days from I-IFSP to Exit', 'Exit Date', 'Exit Reason', 'Exit Note']]

ddid_mvp_filtered = ddid_mvp[ddid_mvp['Exit Note'].notnull()]
pd.set_option('display.max_colwidth', None)
ddid_mvp_filtered.tail()

ddid_mvp_filtered.loc[ddid_mvp_filtered['Exit Note'].str.contains(r"(\d+/\d+/\d+)", case=False)]

In [ ]:
ddid_mvp_filtered = ddid_mvp_filtered.loc[ddid_mvp_filtered['Exit Note'].str.contains('reopen[a-z]*', flags=re.I, regex=True)]

In [ ]:
ddid_mvp_filtered['Reopened Date'] = ddid_mvp_filtered['Exit Note'].str.extract(r'Reopened: (\d{2}/\d{2}/\d{4})')
ddid_mvp_filtered

In [ ]:
ddid_mvp_filtered['rExit Date'] = ddid_mvp_filtered['Exit Note'].str.extract(r'Exit Date: (\d{2}/\d{2}/\d{4})')
ddid_mvp_filtered

In [ ]:
ddid_mvp_filtered['reopen'] = pd.to_datetime(ddid_mvp_filtered['Reopened Date'])
ddid_mvp_filtered

In [ ]:
ddid_mvp_filtered['exit'] = pd.to_datetime(ddid_mvp_filtered['rExit Date'])
ddid_mvp_filtered

In [ ]:
ddid_mvp_filtered['difference'] = ddid_mvp_filtered['reopen']-ddid_mvp_filtered['exit']
ddid_mvp_filtered

In [ ]:
for index, row in ddid_mvp_filtered.iterrows():
    if ((row['exit'] > row['Initial IFSP Date']) and
        (row['exit'] < row['Exit Date']) and
        (row['reopen'] > row['Initial IFSP Date']) and
        (row['reopen'] < row['Exit Date'])):
        diff_days = (row['reopen'] - row['exit']).days
        ddid_mvp_filtered.at[index, 'diff_days'] = diff_days
    else:
        ddid_mvp_filtered.at[index, 'diff_days'] = np.nan


In [ ]:
ddid_mvp_filtered

In [ ]:
newexit = pd.to_datetime(ddid_mvp_filtered['Exit Note'].str.extractall('Exit Date: (\d+.\d+.\d+)')[0]).unstack()
newexit.columns = ['exit_' + str(col + 1) for col in newexit.columns]
newexit

In [ ]:
newenter = pd.to_datetime(ddid_mvp_filtered['Exit Note'].str.extractall('Reopened: (\d+.\d+.\d+)')[0]).unstack()
newenter.columns = ['reopen_'+str(col+1) for col in newenter.columns]
newenter

In [ ]:
open_exit = newexit.join(newenter)
open_exit

In [ ]:
new = ddid_mvp_filtered.join(open_exit)
new

In [ ]:
for index, row in new.iterrows():
    new_diff = 0
    for num in range(1,6):
        if ((row[f'exit_{num}'] != row[f'reopen_{num}']) and
            (row[f'exit_{num}'] != np.nan)  and
            (row[f'reopen_{num}'] != np.nan) and
            (row[f'exit_{num}'] > row['Initial IFSP Date']) and
            (row[f'exit_{num}'] < row['Exit Date']) and
            (row[f'reopen_{num}'] > row['Initial IFSP Date']) and
            (row[f'reopen_{num}'] < row['Exit Date'])):
            mult_diff_days = (row[f'reopen_{num}'] - row[f'exit_{num}']).days
            new_diff += mult_diff_days
            new.at[index, 'diff_days_new'] = new_diff
        else:
            pass
new

In [ ]:
new = new.loc[new['reopen_1'].notnull()]
new

In [ ]:
new.loc[new.diff_days_new<0]

In [ ]:
new['days_in_program'] = new['Days from I-IFSP to Exit']-new['diff_days_new']
new

In [ ]:
ddid['Exit Reason'].value_counts()

In [ ]:
percentages = pd.concat([ddid['Exit Reason'].value_counts(), 
                ddid['Exit Reason'].value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage'))
percentages

In [ ]:
Reason = ['Part B eligibility not determined', 'Parent withdraw', 'Part B eligible', 'Attempts to contact unsuccessful', 'Not eligible for Part B – Exit with referral to other programs', 'Moved out of state', 'Not eligible for Part B – Exit with no referrals', 'Completion of IFSP or no longer requires services', 'Deceased']
Count = [1958, 1243, 1173, 566, 245, 138, 162, 57, 10]
  
plt.bar(Year, Unemployment_Rate)
plt.title('Count of Exit Reason')
plt.xlabel('Reason')
plt.ylabel('Count')
plt.show()

In [ ]:
percentages.to_excel("reasons_chart.xlsx")